## Analyze an Execution plan

TODO: get the plan dynamically
Pre-req: pip install tabulate

In [1]:
import json
from tabulate import tabulate

# sql
sql_command = """SELECT * 
FROM postgres_air.account
WHERE last_name = 'johns'"""
# The EXPLAIN clause
explain_properties = "explain (verbose, format json)"

# The provided JSON plan
plan_json = '[{"Plan":{"Node Type":"Gather","Parallel Aware":false,"Startup Cost":1000,"Total Cost":5583.95,"Plan Rows":48,"Plan Width":53,"Output":["account_id","login","first_name","last_name","frequent_flyer_id","update_ts"],"Workers Planned":1,"Single Copy":false,"Plans":[{"Node Type":"Seq Scan","Parent Relationship":"Outer","Parallel Aware":true,"Relation Name":"account","Schema":"postgres_air","Alias":"account","Startup Cost":0,"Total Cost":4579.15,"Plan Rows":28,"Plan Width":53,"Output":["account_id","login","first_name","last_name","frequent_flyer_id","update_ts"],"Filter":"(account.last_name = \'johns\'::text)"}]}}]'

# Parse the JSON into a Python object
plan_obj = json.loads(plan_json)

# Extract the values of "Total Cost" and "Plan Rows"
cost = plan_obj[0]['Plan']['Total Cost']
rows = plan_obj[0]['Plan']['Plan Rows']
# Check if the "Actual Rows" property is present in the JSON
actual_rows_present = False
for key in plan_obj[0]['Plan']['Plans'][0]:
    if key == 'Actual Rows':
        actual_rows_present = True
        break

# Determine the plan type based on whether the "Actual Rows" property is present
plan_type = 'Actual' if actual_rows_present else 'Estimated'

# Store the results in a dictionary
data = {'Property Name': ['cost', 'plan_rows', 'plan_type'], 'Property Value': [cost, rows, plan_type]}

# Convert the dictionary to a table and print it
print(tabulate(data, headers='keys', tablefmt='pipe'))



| Property Name   | Property Value   |
|:----------------|:-----------------|
| cost            | 5583.95          |
| plan_rows       | 48               |
| plan_type       | Estimated        |
